In [2]:
# Import necessary libraries
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10,7)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

from tensorflow.keras.utils import to_categorical

In [21]:
# Load the dataset 'distances.csv' using pandas' read_csv function.
#distances = pd.read_csv(r'C:\Users\jessi\EE250_FinalProject_Talise_Jessica\distances.csv')
distances = pd.read_csv('distances.csv')
distances.tail(10)

FileNotFoundError: [Errno 2] No such file or directory: 'distances.csv'

In [ ]:
# Visualize the data distribution using a scatterplot
sns.scatterplot(x="dist1", y="dist2", data=distances, hue="location")
plt.title("Scatterplot of Distances and Locations")
plt.show()

In [ ]:
# [STUDENT SECTION: Prepare the data for training and testing]
# 1. Data Preparation:
# Prepare the input features (X)
X = distances[["dist1", "dist2"]].to_numpy()

# Convert the denomination values (1 and 2) to binary labels (0 and 1) for binary classification.
# Hint: Subtract 1 from the denomination values to convert them to 0 and 1.
y = distances[["location"]].to_numpy() # Change this

#y = y - 1
y = to_categorical(y)   # COnverts to one-hot encoding

# Split the data into training and testing sets using train_test_split.
# Use an 80/20 split for training and testing and random_state of 42.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Set the arguments for train_test_split

# Normalize the input test and train features using StandardScaler to standardize reflectance and weight values.
scaler = StandardScaler()
# Use the scaler's fit_transform and transform methods to standardize the training and testing features.
#  (See https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [ ]:
# [STUDENT SECTION: Define and compile the model]
# 2. Instantiate and Train the Model
# - Build a neural network model using TensorFlow/Keras.
# - The model should have:
#    - An input layer with 2 input features (reflectance and weight).
#    - 1 or 2 hidden layers with the number of neurons at your discretion (use 'relu' activation).
#      (Experiment to find the best accuracy)
#    - An output layer with 1 neuron and 'sigmoid' activation for binary classification.
model = Sequential()
# The next line gives an error, but the example code has the same error
model.add(Dense(12, activation='relu', input_shape=(2,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# - Compile the model using Adam optimizer with a learning rate of 0.001,
#   binary_crossentropy as the loss function, and 'accuracy' as the metric.
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#the above has nice accuracy


# - Train the model using X_train and y_train.
#    - Set validation_split to 0.2.
#    - Use EarlyStopping with patience=5 to prevent overfitting.
#    - Train the model for 100 epochs and use a batch size of 32.
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# call the appropriate function to train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)


**#experiement with the accuracy if used only 1 middle layer vs. 2 and so on**

In [ ]:
# [STUDENT SECTION: Evaluate the model]
# Hint:
# - Call model.evaluate() on the test set (X_test, y_test) and print the test accuracy.
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy = %.2f' %(accuracy*100))

In [ ]:
# 4. Plot the Decision Boundary:
# - Define a function to plot the decision boundary of the trained model. This function should take in the feature matrix X, the labels y, and the trained model.
# - This function should:
#    - Generate a mesh grid over the feature space (reflectance and weight).
#    - Use the model to predict labels for each point in the mesh grid.
#    - Plot the decision boundary using contourf.
#    - Overlay the training data points using seaborn's scatterplot.
def plot_decision_boundary(X, y, model):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = np.round(Z).reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.coolwarm)
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y.flatten(), palette='bright', edgecolor="k")
    plt.xlabel('Distance 1')
    plt.ylabel('Distance 2')
    plt.title('Decision Boundary of the Multi-Class Classifier')
    plt.show()


# [STUDENT SECTION: Call the function to plot the decision boundary by passing the correct arguments]
plot_decision_boundary(X_train, y_train, model)




In [ ]:
# 5. Save the Trained Model:
# - Save the trained model to a file named 'model.h5' using model.save().
model.save('distmodel.h5')

# [STUDENT SECTION: Save the model]

# 6. Load the Saved Model and Use it to evalue the "test" data:
# - Load the model using keras.models.load_model and re-evaluate it on the test data.
new_model = load_model('distmodel.h5')
test_loss, test_accuracy = new_model.evaluate(X_test, y_test)
print('Accuracy = %.2f' %(test_accuracy*100))


In [ ]:
# 7. [STUDENT SECTION: Evaluate the loaded model on the test data]
# the accuracy of the loaded model on the test data.  Examine the output
# of predict and think about how to compute the accuracy of the predictions using
# the y_test data.  Hint:  You may need to round the predictions to 0 or 1 using np.round()
# When you use model.predict() in a binary classification problem, the output is usually
# a 2D array where each element is a list containing a single predicted probability
# (e.g., [[0.1], [0.9], [0.3], ...]). To compare these predictions to your 1D y_test array
# (e.g., [0, 1, 0, ...]), you need to "flatten()" the 2D array into a 1D array.


# print the accuracy of the loaded model on the test data

# Evaluate the model on the test data using predict
predictions = new_model.predict(X_test)
predictions = np.round(predictions)
predictions = predictions.flatten()

# count how many are wrong
count = 0
for i in range(len(predictions)):
    if predictions[i] != y_test[i]:
        count += 1

accuracy = 100 * (len(predictions) - count) / len(predictions)
print('Accuracy = %.2f' %(accuracy))

